<a href="https://colab.research.google.com/github/efthymios19/MIRC_EcosystemApproach_MountainousRegion/blob/main/Notebooks/Infiltration_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Field saturated conductivity calculation based on:
"Rapid Measurement of Field-Saturated Hydraulic Conductivity for Areal Characterization" , doi: 10.2136/vzj2007.0159

- b: is the ring radius (cm.)
- d: is the ring insertion depth (cm.)
- D: the depth of ponding (in our case 2.54 cm.)
- parameter lambda (λ) reciprocal of alpha (α) gardner parameter
- C1 and C2 empirically determined constants 0.993 and 0.578 respectively
- LG=C1xd + C2xb the use of this equation requires that the infiltration occur in soil initially dry enough that the hydraulic conductivity is much smaller than Kfs (field saturated conductivity)
- The value of $\lambda$ which is an index of how strongly water is driven by capillary forces in a particular soil, is most strictly determined
by computing an integral of soil's unsaturated hydraulic conductiivy over the range of matric pressure experienced by the soil during the wetting process. The calculation cannot be done as we don't have the value of unsaturated hydraulic conductivity. In the publication "Hydraulic Conductivity Measurements in the Unsaturated Zone Using Improved Well Analyses" pointed out that the sensivity of of conductivity calculations to the value of $\lambda$ isslight and that it would normally be adequte to choose a value from one of the four broad soil categories based on textural and structural considerations.
   1. Soils with significant structural development $\lambda$: 0.08 m.
   2. Coarse and gravelly soils $\lambda$: 0.03 m.
   3. Fine textured soils without macropore $\lambda$: 0.25 m.

For more information on $\lambda$ values see: White, I., and M. J. Sully (1987), Macroscopic and microscopic capillary length and time scales from field infiltration, Water Resour. Res., 23(8), 1514–1522, doi:10.1029/WR023i008p01514.





In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
### Create dataframe with measurements

observation = {
    "Observation": 1,
    "Date": "2026-02-12",
    "X_coordinate": 476901.8,
    "Y_coordinate": 4.202558e6,
    "Temperature": 16.0,
    "Moisture(cm3/cm3)": 15.0,
    "Insertion_depth_cm": 4.0,
    "Wet_area_X_cm": 5.0,
    "Wet_area_Y_cm": 6.0,
    "Notes": "Κορεσμένο έδαφος"
}

# Measurement times (minutes)
measurement_times = [0.55, 2.46, 4.48, 4.46, 5.22]

df = pd.DataFrame({
    "Observation": observation["Observation"],
    "Date": observation["Date"],
    "X_coordinate": observation["X_coordinate"],
    "Y_coordinate": observation["Y_coordinate"],
    "Temperature": observation["Temperature"],
    "Moisture(cm3/cm3)": observation["Moisture(cm3/cm3)"],
    "Insertion_depth_cm": observation["Insertion_depth_cm"],
    "Wet_area_X_cm": observation["Wet_area_X_cm"],
    "Wet_area_Y_cm": observation["Wet_area_Y_cm"],
    "Notes": observation["Notes"],

    # measurement index
    "Measurement_seq": range(1, len(measurement_times)+1),

    # measurement time column
    "tf_min": measurement_times
})

df






In [ ]:
def calculate_Kfs(LG, tf, D0, lambda_val):
    """Equation (12): Kfs = LG/tf * ln(1 + D0/LG + λ)"""
    return (LG / tf) * np.log(1 + (D0 / LG) + lambda_val)


In [ ]:
insertion_depth= df["Insertion_depth_cm"][0]/100
print(insertion_depth)
# Calculate geometry parameter LG
LG = 0.316*np.pi * insertion_depth + 0.184*np.pi * 7.5/100  # Assuming 7.5cm is the ring radius
print(LG)

# Water depth in the ring
D0=2.54/100 #cm constant parameter

lambda_val=0.03

df["Kfs(m/min)"] = calculate_Kfs(
    LG=LG,
    tf=df["tf_min"],
    D0=D0,
    lambda_val=lambda_val
)

# Convert Kfs into m/day (example conversion)
df["Kfs(m/day)"] = df["Kfs(m/min)"] * 1440

df


In [ ]:
# Plotting the results

plt.figure(figsize=(7,4))

plt.scatter(
    df["Measurement_seq"],
    df["Kfs(m/day)"],
    s=90,
    alpha=0.8
)


plt.yscale("log")
plt.xlabel("Measurement --->")
plt.ylabel("Field Saturated Conductivity (Kfs m/day)")
plt.title("Kfs per Measurement Sequence")
plt.grid(True, which="both", ls="--", alpha=0.3)
plt.xticks(df["Measurement_seq"])
plt.legend()
plt.tight_layout()
plt.show()
